In [3]:
from modules.goodreads import Goodreads
from modules.notion import Notion
import os

from dotenv import load_dotenv

load_dotenv()
NOTION_API_KEY = os.environ["NOTION_API_KEY"]
NOTION_DATABASE_ID = os.environ["NOTION_DATABASE_ID"]

# Provide the Goodreads URL to capture the metadata for
URL = "https://www.goodreads.com/book/show/26530386-the-vital-question"

goodreads = Goodreads(URL)

book = goodreads.get_metadata(verbose=True)

Name: The Vital Question: Energy, Evolution, and the Origins of Complex Life
Image: https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1444265552i/26530386.jpg
Format: Paperback
Number of pages: 368
Language: English
ISBN: 9780393352979
Rating: 4.17
Number of ratings: 4732
Number of reviews: 594
Authors: ['Nick Lane']


In [4]:
notion = Notion(NOTION_API_KEY, NOTION_DATABASE_ID)
notion.post_book_from_goodreads(book)

<Response [200]>